# Umsetzung des EfficientNet Klassifizierers.

In [ ]:
!pip install fastai wwf timm -q --upgrade

In [ ]:
from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer

In [ ]:
# Mit drive verbinden zum laden der Bilder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from wwf.utils import state_versions
# state_versions(['fastai', 'fastcore', 'wwf', 'timm'])

# Laden der Daten

In [ ]:
path = Path('/content/drive/MyDrive/Seminar/fish_1')

In [ ]:
# setzen der Transformationen
batch_tfms = [*aug_transforms(size=224, flip_vert=True, min_scale=0.75, max_warp=0), 
              Normalize.from_stats(*imagenet_stats)]
bs=32

In [ ]:
# DataLoaders mit Transformationen erstellen
fish = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460), 
    batch_tfms=batch_tfms)
dls = fish.dataloaders(path, bs=bs)

In [ ]:
# Test der DataLoaders
dls.show_batch(max_n=3)

In [ ]:
# Test der Transformationen
dls.show_batch(nrows=2, ncols=4, unique=true)

# Modell Import
Für die Umsetzung wird Ross Wightman's [repository](https://github.com/rwightman/pytorch-image-models) verwendet. 

In [ ]:
!pip install timm

In [ ]:
from timm import create_model

In [ ]:
net = create_model('efficientnet_b3a', pretrained=True)

In [ ]:
# Analyse der Struktur von FastAi
learn = cnn_learner(dls, resnet18)

In [ ]:
learn.model[-1]

In [ ]:
# vergleich mit der EfficientNet Struktur
net

Anpassen des EfficientNet Models an die FastAi Anforderungen.

In [ ]:
# 
# Quelle der Funktion gemäs Müller 2020: 
# Müller, Zachary (2020): A walk with fastai2. 05_EfficientNet_and_Custom_Weights. 
# Online verfügbar unter https://colab.research.google.com/github/walkwithfastai/
# walkwithfastai.github.io/blob/master/nbs/course2020/vision/05_EfficientNet_and
# _Custom_Weights.ipynb#scrollTo=LyZbSn3OcT0S, zuletzt geprüft am 19.06.2021.
def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

In [ ]:
# erstellen des EfficientNet Modells
# extrahieren der Body Layer
body = create_timm_body('efficientnet_b3a', pretrained=True)
# extrahieren der Anzahl an Input Features
nf = num_features_model(nn.Sequential(*body.children()))
# erstellen des Heads mit entsprechender Anzahl input und output features
head = create_head(nf, dls.c)
# zusammenfügen 
model = nn.Sequential(body, head)
# initialisieren 
apply_init(model[1], nn.init.kaiming_normal_)

Das EfficientNet Model befindet sich nun im von FastAi notwendigen Status und kann normal verwendet werden.

# Umsetzung mit EfficientNet

In [ ]:
learn = Learner(dls, model)

In [ ]:
learn.summary()

In [ ]:
# splitten des Modells um das Einfrieren zu ermoeglichen
learn = Learner(dls, model, splitter=default_split, metrics=error_rate)

In [ ]:
learn.summary()

stage-1

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(15, 1.2e-3)

In [ ]:
# speichern
# learn.save('stage-1-effNet-15ep')

In [ ]:
learn.recorder.plot_loss()

stage-2

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3,  lr_max=slice(1e-5,3e-4))

In [ ]:
# speichern und exportieren
# learn.save('stage-2-effNet-3ep')
# learn.export()

# Zusatz
In diesem Abschnitt sind einige Zusatzfunktionen zur Interpretation der Ergebnisse zu einfachen Umsetzung hinterlegt, identisch zu ResNet.

In [ ]:
# weitere Auswertungen falls gewünscht
# learn34 = ResNet34 Learner
# learn101 = ResNet101 Learner
interp = ClassificationInterpretation.from_learner(learn34)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
# fehlerhaft klassifizierte Bilder anzeigen
interp.plot_top_losses(5, nrows=5)

In [ ]:
# extrahieren der exogenen und endogenen Variablen
x,y = dls.one_batch()

In [ ]:
# auslesen der Klassenbezeichnung der endogenen Variablen
dls.vocab

In [ ]:
y

In [ ]:
# auslesener der Klassenwahrscheinlichkeiten des ersten Bildes
preds,_ = learn34.get_preds(dl=[(x,y)])
preds[0]

In [ ]:
# prüfe die Summe der Wahrscheinlichkeiten
len(preds[0]), preds[0].sum()